# Ramsey King
# DSC 680 - Applied Data Science
# April 9 - 2022

###  This Jupyter file will pull the talk information down from the websites speeches.byu.edu and www.churchofjesuschrist.org/study/general-conference/speakers

In [1]:
# Retrieve talks by speaker on speeches.byu.edu using BeautifulSoup

from bs4 import BeautifulSoup as bs
import requests
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
# Get list of urls for each of the topics
# For my reference, number of talks by speaker:  Nelson - 22, Maxwell - 30, Holland - 27, Hinckley - 41, Monson - 14, Oaks - 35, Eyring - 25 (Total - 194)
url = "https://speeches.byu.edu/speakers/"

url_list = [
    'russell-m-nelson/', 'neal-a-maxwell/', 'jeffrey-r-holland/', 'gordon-b-hinckley/', 'thomas-s-monson/',
    'dallin-h-oaks', 'henry-b-eyring'
]

links = []
new_links = []

# some of the links are repeated with '?M=V' and '?M=A' and those will be removed so we can have a unique set of links to pull talk information from.

for u in url_list:
    response = requests.get(url+u)
    html = response.content
    video = '?M=V'
    soup = bs(html, 'lxml')
    for link in soup.findAll("a", attrs={'href': re.compile("^https://speeches.byu.edu/talks/[\D+]")}):
        links.append(link.get('href').replace(video, ''))

    audio = '?M=A'
    for link in links:
        new_link = link.replace(audio, '')
        new_links.append(new_link)
    new_links = list(set(new_links))



In [3]:
len(new_links)

195

In [4]:
# We may have one duplicate, so something to be aware when we pull the talk data.

'''Retrieves talk text from each of the urls using beautiful soup.  Information such as the talk/sermon text, title, topic, date, and speaker will be retreived
and put into a dataframe.  A csv file will also be saved due to the time it takes to run or in case we need to work offline.'''
talk_text = []
title_text = []
topics = []
dates = []
speakers = []
speaker_position = []

for link in new_links:
    url = link
    response = requests.get(url)
    html = response.content
    soup = bs(html, 'lxml')
    temp_list = []

    link_splitter = link.split('/')
    speakers.append(link_splitter[4].replace('-', ' '))

    result = soup.find('p', class_='single-speech__speaker-subtext single-speech__speaker-position')
    speaker_position.append(result.text if result else 'Title Not Found')

    for div in soup.find_all('div', class_='single-speech__content'):
        talk_text.append(div.get_text(strip=True))
    for h1 in soup.find_all('h1', class_='single-speech__title'):
        title_text.append(h1.get_text(strip=True))
    for p in soup.find_all('p', class_='single-speech__speaker-subtext single-speech__date'):
        dates.append(p.get_text(strip=True))
    for p in soup.find_all('p', class_='single-speech__related-tag'):
        temp_list.append(p.get_text(strip=True))
        temp_str = ','.join(temp_list)
    topics.append(temp_str)


dict = {'titles': title_text , 'speaker': speakers, 'speaker_position': speaker_position , 'date': dates, 'talks': talk_text, 'topics': topics, }
df = pd.DataFrame(dict)
df.to_csv('talks.csv')

df.head()

,titles,speaker,speaker_position,date,talks,topics
0,Nailing Our Colors to the Mast,jeffrey r holland,President of Brigham Young University,"September 10, 1985",Man-made MagicWe live in a most remarkable age...,Self-Improvement
1,"“Mirror, Mirror, on the Wall”: A Look at the “...",jeffrey r holland,Commissioner of the Church Educational System,"February 20, 1979","Thank you, President Oaks, and all of you. My ...","Pride,Selflessness"
2,“And Be Not Faithless But Believing”,gordon b hinckley,Title Not Found,"January 3, 1962",The Lord has commanded us to “be not faithless...,"Faith,Jesus Christ"
3,“Cast Not Away Therefore Your Confidence”,jeffrey r holland,of the Quorum of the Twelve Apostles,"March 2, 1999",There is a lesson in the Prophet Joseph Smith’...,"Faith,Hope,Revelation,Podcast: Classic Speeches"
4,Be Thou an Example of the Believers,thomas s monson,of the First Presidency,"October 15, 1991",You have been given a great trust. You have th...,Example


In [20]:
# Now to pull information from the church general conference website.

# Get list of urls for each of the topics
# For my reference, number of talks by speaker:  Nelson - , Maxwell - , Holland - , Hinckley - , Monson - , Oaks - , Eyring -  (Total - )
gc_url = "https://www.churchofjesuschrist.org/study/general-conference/speakers/"

gc_url_list = [
    'russell-m-nelson/', 'neal-a-maxwell/', 'jeffrey-r-holland/', 'gordon-b-hinckley/', 'thomas-s-monson/',
    'dallin-h-oaks', 'henry-b-eyring'
]

links = []
# new_links = []

for u in gc_url_list:
    response = requests.get(gc_url+u)
    html = response.content
    soup = bs(html, 'lxml')
    for link in soup.findAll("a", attrs={'href': re.compile("^/study/general-conference/\w+/(?:\w+/)(?:\w+)")}):
        links.append(link.get('href'))

    '''audio = '?M=A'
    for link in links:
        new_link = link.replace(audio, '')
        new_links.append(new_link)
    new_links = list(set(new_links))'''
print(links)

['/study/general-conference/2021/10/59nelson', '/study/general-conference/2021/10/47nelson', '/study/general-conference/2021/10/11nelson', '/study/general-conference/2021/04/57nelson', '/study/general-conference/2021/04/49nelson', '/study/general-conference/2021/04/36nelson', '/study/general-conference/2021/04/11nelson', '/study/general-conference/2020/10/58nelson', '/study/general-conference/2020/10/46nelson', '/study/general-conference/2020/10/37nelson', '/study/general-conference/2020/10/11nelson', '/study/general-conference/2020/04/57nelson', '/study/general-conference/2020/04/46nelson', '/study/general-conference/2020/04/45nelson', '/study/general-conference/2020/04/37nelson', '/study/general-conference/2020/04/11nelson', '/study/general-conference/2019/10/57nelson', '/study/general-conference/2019/10/46nelson', '/study/general-conference/2019/10/36nelson', '/study/general-conference/2019/10/24nelson', '/study/general-conference/2019/04/57nelson', '/study/general-conference/2019/0